In [1]:
#importando as bibliotecas
import pandas as pd
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials

Optei por usar um input para definir o nome da pasta onde estarão os dados porque assim eu posso trabalhar com multiplas pastas de dados sem precisar ficar alterando o código

In [ ]:
#definindo onde as planilhas com as faturas estão localizadas
path = os.getcwd() +'/' + input('Insira o nome da sua pasta:')
#print para ver se você inseriu a pasta correta
print(path)

As planilhas de fatura do nubank são geradas indivualmente, mês a mês. Então, eu preciso concatenar elas todas em uma única planilha

In [3]:
#escolhendo quais arquivos serão lindos
files = [file for file in os.listdir(path) if not file.startswith('.')]

#criando um banco de dados vazio
all_months_df = pd.DataFrame()

#concatenando as planilhas lidas dentro do banco de dados vazios
for file in files: 
    current_df = pd.read_csv(path+"/"+file)
    all_months_df = pd.concat([all_months_df, current_df])
    
#salvando essa nova planilha
all_months_df.to_csv("nu_df.csv", index=False)

Agora que essa planilha única foi salva no meu dispositivo, eu posso ler ela com o python :) 

In [ ]:
#lendo essa nova planilha
nu_df = pd.read_csv('nu_df.csv')
#observand os dados para encontrar possíveis problemas
nu_df.head()

# 1. Corrigindo problemas

Ao ler a planilha detectei que os pagamentos de fatura estavam todos com NaN na coluna 'category'. Então eu optei por excluir estas linhas, já que o que me interessa na fatura são os gastos

In [5]:
#apagando as linhas com NaN
nu_df = nu_df.dropna()
#observando ser esses problemas foram corrigidos
nu_df.head()

,date,category,title,amount
0,2020-12-12,serviços,Picpay *Vt9 Pgtoconta 6/6,13.52
3,2020-12-16,educação,Memrise.Com,155.68
5,2020-12-21,transporte,Uberbr Uber * Pending,11.39
7,2020-12-21,transporte,Uberbr Uber * Pending,16.73
8,2020-12-24,eletrônicos,Mercpago*Clickbus,50.75


## 1.1 Me certificando de que os dados estão com os tipos corretos

In [6]:
#transformando os valores da coluna 'amount' em números
nu_df['amount'] = pd.to_numeric(nu_df['amount'])

Para facilitar análises posteriores eu resolvi separar a data em mais duas colunas: month e year

In [7]:
#criando duas colunas de data novas, uma para mês e uma para o ano
nu_df ['month'] = pd.to_datetime(nu_df['date']).dt.month
nu_df ['year'] = pd.to_datetime(nu_df['date']).dt.year
nu_df.head()

,date,category,title,amount,month,year
0,2020-12-12,serviços,Picpay *Vt9 Pgtoconta 6/6,13.52,12,2020
3,2020-12-16,educação,Memrise.Com,155.68,12,2020
5,2020-12-21,transporte,Uberbr Uber * Pending,11.39,12,2020
7,2020-12-21,transporte,Uberbr Uber * Pending,16.73,12,2020
8,2020-12-24,eletrônicos,Mercpago*Clickbus,50.75,12,2020


# 2. Importando os dados para uma planilha do GoogleSheets

Eu queria que minha visualização dos dados fosse toda feita através do Google DataStudio, por isso, resolvi fazer com que essa planilha atualizasse uma planilha do GoogleSheets, assim eu poderia apenas importar os dados para o DataStudio e assim montar a visualização

In [8]:
#conectando com a API do google

scope = scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

#conectando a credencial de verificação do google
creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/pcfmp/OneDrive/Documentos/PY/proje_nu.json', scope)

#autorizando o login por parte do cliente
client = gspread.authorize(creds)

#abrindo a planilha no GoogleSheets e escolhendo a instância(aba) correta dela
sheet = client.open('nubank_gastos_automação')
sheet_instance = sheet.get_worksheet(0)


In [ ]:
#atualizando os dados da planilha do GoogleSheets com a planilha criada com os dados locais
sheet_nu = sheet_instance.update([nu_df.columns.values.tolist()] + nu_df.values.tolist())

Pronto, agora é só eu abrir o google DataStudio e conectar ele a essa planilha do Sheets :)